# Session 10 - MDP Planning

You need to read the theory lectures before practicing with this notebook:
- [Preclass S10](https://hackmd.io/@KylePaul/ML_Preclass_S10)
- [Slide](https://hackmd.io/@KylePaul/ML_Slide_S10)

```{contents}

```



# Excercise: Frozen Lake


**Description:** Game glacial lake, is a rectangle of many squares, each of which will be ice or iceless. If you go into the ice-free box, the game over! Your task is to move from S to G.

![frozen-lake-description](https://i.imgur.com/jLhoSev.png)
![frozen-lake-map](https://i.imgur.com/1WS7wFJ.png)


**Q-values table**

|State(cell)|Left|Down|Right|Up|
|---|---|---|---|---|
|0 | 0 | 0 | 0 | 0 |
|1 | 0 | 0 | 0 | 0 |
|2| 0 | 0 | 0 | 0 |
|...| 0 | 0 | 0 | 0 |
|15 | 0 | 0 | 0 | 0 |



### Import libraries & define constants

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
GAMMA = 0.9

ACTION_LEFT = 0
ACTION_RIGHT = 1
ACTION_UP = 2
ACTION_DOWN = 3

ACTIONS = ["left", "right", "up", "down"]

### TODO 1: ENVIRONMENT SIMULATION
Class `FrozenLake` is a class that describes the game environment, and also indicates how the environment will respond to each Agent's action
- Complete the function `step` of `class FrozenLake` này.
- The input of this function:
  - `state` is the current position (index) of Agent
  - `action` is the action that Agent takes
- The output of this function
  - `next_state` new position (index) of Agent
  - `reward` is the reward Agent receives
    - `1` if `next_state` is goal
    - `-1` if `next_state` is a hole
    - `0` for other positions
- Note:
  - If the Agent is at the 4 edges of the map and continues to go outside the map or if he is standing at the pits or destination, make the Agent stand still
  - The rest of the cases, Agent moves normally

In [ ]:
class Env:
  def __init__(self, rows, cols, start, goal, holes):
    # YOUR CODE HERE
    self.rows = rows
    self.cols = cols
    self.start = start
    self.goal = goal
    self.holes = holes

  def render(self):
    # render the current environment
    pass

  # get_next_state
  # update
  def step(self, state, action):
    if state == self.goal or state in self.holes:
      return state, 0

    # Boundaries checking
    if state % self.cols == 0 and action == ACTION_LEFT:
      return state, 0
    if state % self.cols == self.cols - 1 and action == ACTION_RIGHT:
      return state, 0
    if state < self.cols and action == ACTION_UP:
      return state, 0
    if state > (self.rows-1) * self.cols - 1 and action == ACTION_DOWN:
      return state, 0

    # Agent moves right, left, up, down
    if action == ACTION_LEFT:
      next_state = state - 1
    elif action == ACTION_RIGHT:
      next_state = state + 1
    elif action == ACTION_UP:
      next_state = state - self.cols
    else:
      next_state = state + self.cols

    # assign reward given the state
    reward = 0
    if next_state in self.holes:
      reward = -1
    elif next_state == self.goal:
      reward = 1

    return next_state, reward

In [ ]:
rows = 4
cols = 4
start = 0
goal = 15
holes = [5,7,11,12]

env = Env(rows, cols, start, goal, holes)
env.step(0, ACTION_DOWN)

(4, 0)

### TODO 2: EXPLORE/INTERACT WITH THE ENVIRONMENT TO FIND THE OPTIMAL POLICY

**Bellman Equations**

$$
Q^*(s,a) = R(s,a) + \gamma \sum_{s'} P(s'|s,a)\max_{a'} Q^*(s',a')
$$


Complete the function `planning` as follows
- Input:
  - `iter`: number of interation loops you want to do
  - `env`: Game environment has been initialized
- Output: matrix Q has been optimized (optimal Q-table)

In [ ]:
Q = np.zeros(shape=(env.rows * env.cols, 4))

In [ ]:
for state in range(Q.shape[0]):
  for action in range(Q.shape[1]):
    next_state, reward = env.step(state, action)

    if next_state != state: # not terminate
      Q[state, action] = reward + GAMMA * np.max(Q[next_state])
Q

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.],
       [ 0.,  0., -1.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [ ]:
def planning(iter, env):
  # state-action value function - initial values are 0
  Q = np.zeros(shape=(env.rows * env.cols, 4))

  for i in range(iter):
    for state in range(Q.shape[0]):
      for action in range(Q.shape[1]):
        # update
        next_state, reward = env.step(state, action)

        # Do not update when the agent is in the hole or goal
        # Do not update when the agent is on the edge and keep going out
        if next_state != state:
          Q[state, action] = reward + GAMMA * np.max(Q[next_state])
  return Q

In [ ]:
# run planning
iter = 1000
Q = planning(iter, env)
print(Q)

[[ 0.        0.59049   0.        0.59049 ]
 [ 0.531441  0.6561    0.       -1.      ]
 [ 0.59049   0.59049   0.        0.729   ]
 [ 0.6561    0.        0.       -1.      ]
 [ 0.       -1.        0.531441  0.6561  ]
 [ 0.        0.        0.        0.      ]
 [-1.       -1.        0.6561    0.81    ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.729     0.59049  -1.      ]
 [ 0.6561    0.81     -1.        0.81    ]
 [ 0.729    -1.        0.729     0.9     ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [-1.        0.9       0.729     0.      ]
 [ 0.81      1.        0.81      0.      ]
 [ 0.        0.        0.        0.      ]]


### TODO 3: OUTPUT TO AGENT'S PATH & ACTION
- Complete the function ``follow_policy``
- Input of this function is
  - ``Q`` is the optimal Q-table (Q matrix after training)
  - ``env`` game environment
- Output of this function
  - ``path`` **List** contains the history of the Agent's path
  - ``path_actions`` **List** contains a history of the agent's actions
- Guildance:
  - Search for the optimal action in each state based on the **Q** matrix then perform the action using the `step` function written above

In [ ]:
def follow_policy(Q, env):
  path = [env.start]
  path_actions = []
  current_state = path[0]

  while True:
    # stop conditions
    if current_state == env.goal:
      break
    if current_state in env.holes:
      break

    # follow the optimal policyt pi
    optimal_action = np.argmax(Q[current_state])
    next_state, reward = env.step(current_state, optimal_action)
    path.append(next_state)
    path_actions.append(ACTIONS[optimal_action])
    current_state = next_state

  return path, path_actions

![frozen-lake-map](https://i.imgur.com/1WS7wFJ.png)


In [ ]:
Q

array([[ 0.      ,  0.59049 ,  0.      ,  0.59049 ],
       [ 0.531441,  0.6561  ,  0.      , -1.      ],
       [ 0.59049 ,  0.59049 ,  0.      ,  0.729   ],
       [ 0.6561  ,  0.      ,  0.      , -1.      ],
       [ 0.      , -1.      ,  0.531441,  0.6561  ],
       [ 0.      ,  0.      ,  0.      ,  0.      ],
       [-1.      , -1.      ,  0.6561  ,  0.81    ],
       [ 0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.729   ,  0.59049 , -1.      ],
       [ 0.6561  ,  0.81    , -1.      ,  0.81    ],
       [ 0.729   , -1.      ,  0.729   ,  0.9     ],
       [ 0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ],
       [-1.      ,  0.9     ,  0.729   ,  0.      ],
       [ 0.81    ,  1.      ,  0.81    ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ]])

In [ ]:
# run follow_policy
follow_policy(Q, env)

([0, 1, 2, 6, 10, 14, 15], ['right', 'right', 'down', 'down', 'down', 'right'])

## Branch and Bound with Backtracking

In [ ]:
import numpy as np

matrix = np.array(
    [[1, 0, 0, 0],
    [0, -1, 0, -1],
    [0, 0, 0, -1],
    [-1, 0, 0, 2]]
)

control_i = [1, 0, -1, 0]
control_j = [0, -1, 0, 1]
n_rows, n_cols = 4, 4
num_step = 0
path = []
best_num_step = n_rows * n_cols

def valid(next_i, next_j):
  if (next_i >= n_rows or next_i < 0 or next_j >= n_cols or next_j < 0 or matrix[next_i][next_j] == -1):
    return False
  return True

def backtracking(cur_i, cur_j, num_step, path, best_num_step):
  if matrix[cur_i][cur_j] == -1:
    return False, num_step, path, best_num_step

  elif matrix[cur_i][cur_j] == 2:
    path.append((cur_i, cur_j))
    if num_step < best_num_step:
      best_num_step = num_step
    return True, num_step, path, best_num_step

  temp = matrix[cur_i][cur_j]
  matrix[cur_i][cur_j] = -1
  path.append((cur_i, cur_j))
  for k in range(len(control_i)):
    next_i = cur_i + control_i[k]
    next_j = cur_j + control_j[k]
    if valid(next_i, next_j):
      result, num_step, path, best_num_step = backtracking(next_i, next_j, num_step + 1, path, best_num_step)
      if result:
        return True, num_step, path, best_num_step

  matrix[cur_i][cur_j] = temp
  path.pop()

  return False, num_step, path, best_num_step

result, num_step, path, best_num_step = backtracking(0, 0, num_step, path, best_num_step)

if result:
    print(path)
    print(len(path))
else:
    print("No pathway")

[(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3)]
7


Using `env.P[state][action]` is updated with the formula:

$$
\forall s,a: ~ Q(s,a) = \sum_{s'} P(s'|s,a)\big[R(s,a,s') + \gamma \max_{a'}(Q(s',a'))\big].
$$

This is the **Bellman equation** for Q-learning, which expresses the optimal Q-value as a function of the transition probabilities, rewards, and future Q-values. This equation can be used to iteratively update the Q-table until convergence.

However, using env.P[state][action] is not a good idea for several reasons. First, it assumes that you have access to the full model of the environment, which is often not the case in real-world problems. Second, it requires you to loop over all possible next states for each state-action pair, which can be very inefficient and slow. Third, it does not account for the stochasticity and uncertainty of the environment, which can lead to suboptimal policies.

A better alternative is to use **sample-based updates**, which use the actual observations from the environment to update the Q-table. This way, you only need to update one state-action pair at a time, based on the reward and the next state that you observe. This is more efficient, realistic, and robust.

The sample-based update rule for Q-learning is:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \big[r + \gamma \max_{a'}(Q(s',a')) - Q(s,a)\big]
$$

where $\alpha$ is the **learning rate** that controls how much you update your Q-value based on new information.

If you don't want to use epsilon-greedy exploration, you can use other exploration strategies, such as **Boltzmann exploration** or **optimistic initialization**. Boltzmann exploration chooses an action based on a probability distribution that depends on the Q-values and a temperature parameter. Optimistic initialization sets the initial Q-values to high values, so that the agent is motivated to explore new actions.

Here is a modified version of your code that uses sample-based updates and Boltzmann exploration:

```python
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True, render_mode="rgb_array")
import numpy as np

terminals = [5, 7, 11, 12, 15]

def planning(env, iters, terminals, gamma=0.9):
  num_state = env.observation_space.n
  num_action = env.action_space.n

  Q = np.zeros(shape=(num_state, num_action))
  for i in range(iters):
    state = env.reset() # reset the environment at the beginning of each episode
    while True:
      action = choose_action(state, Q) # choose an action using Boltzmann exploration
      next_state, reward, done, info = env.step(action) # take the action and observe the next state and reward
      if next_state not in terminals:
        update_Q(state, action, reward, next_state, Q) # update the Q-table using sample-based updates
      else:
        Q[state, action] = reward # terminal states have zero future value
      state = next_state # update the current state
      if done: # check if the episode is over
        break # exit the loop
  return Q

def choose_action(state, Q):
  tau = 0.1 # temperature parameter
  probs = np.exp(Q[state] / tau) / np.sum(np.exp(Q[state] / tau)) # compute the Boltzmann probabilities
  action = np.random.choice(range(len(Q[state])), p=probs) # choose an action based on the probabilities
  return action

def update_Q(state, action, reward, next_state, Q):
  alpha = 0.1 # learning rate
  Q[state][action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action]) # update the Q-value using sample-based updates

def get_optimal_actions(env, optimal_Q):
  optimal_actions = []

  current_state = env.reset() # reset the environment at the beginning of each episode
  while True:
    if current_state in terminals:
      break
    optimal_action = np.argmax(Q[current_state])
    optimal_actions.append(optimal_action)
    next_state, reward, done, info = env.step(optimal_action)
    current_state = next_state
  return optimal_actions

Q = planning(env, 1000, terminals)
optimal_actions = get_optimal_actions(env, Q)
```